In [1]:
%load_ext autoreload
%autoreload 2
# turn off autoreload so that we can use the old model 
# when editing the current project

from coeditor.common import *
import os

os.chdir(proj_root())

In [2]:
from coeditor.retrieval_model import RetrievalEditorModel
from coeditor.api import EditPredictionService, QueryRefEditEncoder, BatchArgs, DecodingArgs

# target_dir = proj_root() / "../SPOT-copy/"
target_dir = proj_root()
model_path = get_model_dir(True) / "coeditor-large-request-stub-v2"
model = RetrievalEditorModel.load(model_path)
model.to("cuda:2")
batch_args = BatchArgs.service_default()
service = EditPredictionService(
    target_dir,
    model,
    batch_args=batch_args,
    encoder=QueryRefEditEncoder(
        max_ref_tks=batch_args.max_ref_tks,
        max_query_tks=batch_args.max_query_tks,
        max_output_tks=batch_args.max_output_tks,
    ),
    # dec_args = DecodingArgs(do_sample=False, num_beams=8, length_penalty=0.0)
    dec_args = DecodingArgs(do_sample=True, top_p=0.95, marginalize_samples=50)
)


In [7]:
service.dec_args = DecodingArgs(do_sample=True, top_p=0.9, marginalize_samples=25)
# service.dec_args = DecodingArgs(do_sample=False, num_beams=8, length_penalty=0.0)
service.tlogger.clear()
service.suggest_edit(Path("src/coeditor/history.py"), 316, apply_edit=False)
service.tlogger.as_dataframe()

respect_lines = 6
stub files: dict_keys(['scripts.coeditor.start_server', 'coeditor.encoding', 'coeditor.retrieval_model', 'coeditor.history'])
num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
========== Sugeestion 0 ==========
score: 0.12
* Modified: 
    ... 23 lines omitted...
========== Sugeestion 1 ==========
score: 0.08
* Modified: 
    ... 2 lines omitted...
              before: PythonProject,
              code_changes: Mapping[ModuleName, str | None],
              symlinks: Mapping[ModuleName, ModuleName] = {},
              remove_comments: bool = True,
              commit_info: "CommitInfo | None" = None,
          ) -> "ProjectEdit":
    -         mod_changes = dict[ModuleName, PythonModule | None]()
    +         mod_changes: Dict[ModuleName, PythonModule] = {}
              for mname, new_code in code_changes.items():
                  if new_code is None:
                      mod_changes[mname] = None
                  else:
  

,name,count,avg_time,total_time
5,run model,1,5.209942,5.209942
3,model.generate,1,5.055078,5.055078
1,construct project edit,1,0.639005,0.639005
4,assemble changes,1,0.144952,0.144952
2,encode edits,1,0.032825,0.032825
0,get target element,1,0.000374,0.000374


In [4]:
service.suggest_edit(Path("src/coeditor/common.py"), 294)

stub files: dict_keys(['coeditor.api', 'coeditor.common', 'coeditor.encoders', 'coeditor.retrieval_model'])
num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
========== Sugeestion 0 ==========
score: -11.35
* Modified: 
        # edit: coeditor.common/random_subset
    -   def random_subset(all, n: int, seed: int = 42):
    +   def random_subset(all, n: int, rng: random.Random | None = None):
    +       if rng is None:
    +           rng = random.Random(42)
            if isinstance(all, Sequence):
                xs = [x for x in all]
    -           random.Random(seed).shuffle(xs)
    +           rng.shuffle(xs)
                return xs[:n]
            elif isinstance(all, Mapping):
                keys = [k for k in all]
    -           random.Random(seed).shuffle(keys)
    +           rng.shuffle(keys)
                return {k: all[k] for k in keys[:n]}
            else:
                raise ArgumentError(all, f"Unsupported arg type: {typ

In [5]:
service.suggest_edit(Path("src/coeditor/encoders.py"), 292)

stub files: dict_keys(['coeditor.api', 'coeditor.common', 'coeditor.encoders', 'coeditor.retrieval_model'])
num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
========== Sugeestion 0 ==========
score: -2.132
* Modified: 
        # edit: coeditor.encoders/has_change
    -   def has_change(tks: TokenSeq) -> bool:
    -       return Add_id in tks or Del_id in tks
    +   def has_change(tks: TokenSeq, skip_prefix: int = 16) -> bool:
    +       return Add_id in tks or Del_id in tks[skip_prefix:]
========== Sugeestion 1 ==========
score: -3.006
* Modified: 
        # edit: coeditor.encoders/has_change
    -   def has_change(tks: TokenSeq) -> bool:
    -       return Add_id in tks or Del_id in tks
    +   def has_change(tks: TokenSeq, skip_prefix: int = 16) -> bool:
    +       return Add_id in tks or Del_id in tks or skip_prefix in tks
========== Sugeestion 2 ==========
score: -3.18
* Modified: 
        # edit: coeditor.encoders/has_change
    -   def h

In [6]:
service.tlogger.as_dataframe()


,name,count,avg_time,total_time
3,run model,3,2.283796,6.851387
1,construct project edit,3,0.674862,2.024587
0,get target element,3,0.257443,0.772328
2,encode edits,3,0.151020,0.453060
